In [1]:
# # in colab
# !pip install vaex -q
# # !pip install vaex.ui -q
# # !pip install IPython==7.0.0 -q
# # !pip install PyQt5 -q
# exit()

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import os, shutil, time, h5py
import gc  # zeruje ram
import vaex
import re

In [3]:
list_file = os.listdir("D:/TAXI/files")
list_file_csv = [i for i in list_file if i.endswith(".csv")]
list_file_csv

['yellow_tripdata_2009-01.csv',
 'yellow_tripdata_2009-02.csv',
 'yellow_tripdata_2009-03.csv',
 'yellow_tripdata_2009-04.csv',
 'yellow_tripdata_2009-05.csv',
 'yellow_tripdata_2009-06.csv',
 'yellow_tripdata_2009-07.csv',
 'yellow_tripdata_2009-08.csv',
 'yellow_tripdata_2009-09.csv',
 'yellow_tripdata_2009-10.csv',
 'yellow_tripdata_2009-11.csv',
 'yellow_tripdata_2009-12.csv',
 'yellow_tripdata_2010-01.csv',
 'yellow_tripdata_2010-02.csv',
 'yellow_tripdata_2010-03.csv',
 'yellow_tripdata_2010-04.csv',
 'yellow_tripdata_2010-05.csv',
 'yellow_tripdata_2010-06.csv',
 'yellow_tripdata_2010-07.csv',
 'yellow_tripdata_2010-08.csv',
 'yellow_tripdata_2010-09.csv',
 'yellow_tripdata_2010-10.csv',
 'yellow_tripdata_2010-11.csv',
 'yellow_tripdata_2010-12.csv',
 'yellow_tripdata_2011-01.csv',
 'yellow_tripdata_2011-02.csv',
 'yellow_tripdata_2011-03.csv',
 'yellow_tripdata_2011-04.csv',
 'yellow_tripdata_2011-05.csv',
 'yellow_tripdata_2011-06.csv',
 'yellow_tripdata_2011-07.csv',
 'yellow

`conwersja plików .csv do .hdf5`

In [16]:
file_with_error = []

for file_from_list in list_test_file:
    print(f"---wczytuje plik {file_from_list}")
    try:
        moment = vaex.read_csv("D:/TAXI/files/"+file_from_list, 
                           copy_index=False, 
                           convert=True,                   # otwieracjąc convertuje na hdf5 
                           chunk_size=1_000_000,  # wielkość pojedyńczogo chunk (rows)
                           low_memory=False)        # podczas łączenia chunks może dość do problemów w typach kolumn. Dzieki False unikamy problemu
        print("---exportuje do h5")
        del moment       # usuwa zmienną
        gc.collect()         # czyści RAM
        gc.collect()         # czyści RAM 
        
    except pd.errors.ParserError:        # jeżli błąd to dodaj nazwę pliku do listy
        file_with_error.append(file_from_list)



---wczytuje plik yellow_tripdata_2010-02.csv
---wczytuje plik yellow_tripdata_2010-03.csv
---wczytuje plik yellow_tripdata_2010-04.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-05.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-06.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-07.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-08.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-09.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-10.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-11.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2010-12.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2011-01.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2011-02.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2011-03.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2011-04.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2011-05.csv
---exportuje do h5
---wczytuje pl

---exportuje do h5
---wczytuje plik yellow_tripdata_2020-11.csv
---exportuje do h5
---wczytuje plik yellow_tripdata_2020-12.csv
---exportuje do h5


`pliki których nie udało się otworzyć`

In [9]:
file_with_error 
# 'yellow_tripdata_2010-02.csv', 'yellow_tripdata_2010-03.csv'

['yellow_tripdata_2010-02.csv', 'yellow_tripdata_2010-03.csv']

`sprawdzmy czy wszystkie pliki mają te same kolumny`
vaex - get_column_names()

In [12]:
%%time
df1 = vaex.open('D:/TAXI/files/yellow_tripdata_.csv.hdf5')
based_columns = df1.get_column_names()
based_columns

Wall time: 131 ms


['vendor_name',
 'Trip_Pickup_DateTime',
 'Trip_Dropoff_DateTime',
 'Passenger_Count',
 'Trip_Distance',
 'Start_Lon',
 'Start_Lat',
 'Rate_Code',
 'store_and_forward',
 'End_Lon',
 'End_Lat',
 'Payment_Type',
 'Fare_Amt',
 'surcharge',
 'mta_tax',
 'Tip_Amt',
 'Tolls_Amt',
 'Total_Amt']

In [13]:
%%time

all_columns = {}
differences_columns = {}

for file in [ i for i in list_file_csv if i not in file_with_error ]:  # przeszukaj wszystkie pliki poza tymi które spowodowały wcześniejszy błąd
    print("-------Plik: ", file)
    df = vaex.open('D:/TAXI/files/' + file + '.hdf5')
    all_columns[file] = df.get_column_names()
    try:
        df = df[based_columns]
        print("OK")  
    except NameError:
        differences_columns[file] = df.get_column_names()
        print("--inne kolumny")  

-------Plik:  yellow_tripdata_2009-01.csv
OK
-------Plik:  yellow_tripdata_2009-02.csv
OK
-------Plik:  yellow_tripdata_2009-03.csv
OK
-------Plik:  yellow_tripdata_2009-04.csv
OK
-------Plik:  yellow_tripdata_2009-05.csv
OK
-------Plik:  yellow_tripdata_2009-06.csv
OK
-------Plik:  yellow_tripdata_2009-07.csv
OK
-------Plik:  yellow_tripdata_2009-08.csv
OK
-------Plik:  yellow_tripdata_2009-09.csv
OK
-------Plik:  yellow_tripdata_2009-10.csv
OK
-------Plik:  yellow_tripdata_2009-11.csv
OK
-------Plik:  yellow_tripdata_2009-12.csv
OK
-------Plik:  yellow_tripdata_2010-01.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-04.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-05.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-06.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-07.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-08.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-09.csv
--inne kolumny
-------Plik:  yellow_tripdata_2010-10.csv
--inne kolumny
----

`spradzmy nazwy kolumn w poszególnych latach`

In [29]:
schematy = {}

for i in [i for i in all_columns if i.endswith(".csv")]:
    print(i, len( all_columns[i]), all_columns[i])
    schematy[i] = all_columns[i]
    

yellow_tripdata_2009-01.csv 18 ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code', 'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt']
yellow_tripdata_2009-02.csv 18 ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code', 'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt']
yellow_tripdata_2009-03.csv 18 ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code', 'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt']
yellow_tripdata_2009-04.csv 18 ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTi

In [31]:
schematy

{'yellow_tripdata_2009-01.csv': ['vendor_name',
  'Trip_Pickup_DateTime',
  'Trip_Dropoff_DateTime',
  'Passenger_Count',
  'Trip_Distance',
  'Start_Lon',
  'Start_Lat',
  'Rate_Code',
  'store_and_forward',
  'End_Lon',
  'End_Lat',
  'Payment_Type',
  'Fare_Amt',
  'surcharge',
  'mta_tax',
  'Tip_Amt',
  'Tolls_Amt',
  'Total_Amt'],
 'yellow_tripdata_2009-02.csv': ['vendor_name',
  'Trip_Pickup_DateTime',
  'Trip_Dropoff_DateTime',
  'Passenger_Count',
  'Trip_Distance',
  'Start_Lon',
  'Start_Lat',
  'Rate_Code',
  'store_and_forward',
  'End_Lon',
  'End_Lat',
  'Payment_Type',
  'Fare_Amt',
  'surcharge',
  'mta_tax',
  'Tip_Amt',
  'Tolls_Amt',
  'Total_Amt'],
 'yellow_tripdata_2009-03.csv': ['vendor_name',
  'Trip_Pickup_DateTime',
  'Trip_Dropoff_DateTime',
  'Passenger_Count',
  'Trip_Distance',
  'Start_Lon',
  'Start_Lat',
  'Rate_Code',
  'store_and_forward',
  'End_Lon',
  'End_Lat',
  'Payment_Type',
  'Fare_Amt',
  'surcharge',
  'mta_tax',
  'Tip_Amt',
  'Tolls_Amt',

In [47]:
pd.DataFrame(schematy["yellow_tripdata_2009-01.csv"]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt


In [352]:
dfc =  pd.DataFrame()

for i in schematy.keys():
    df_new = pd.DataFrame(schematy[i]).T
    df_new["file_name"] = i
    dfc = pd.concat([dfc, df_new], axis=0, ignore_index=True)
    
dfc.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,file_name,18,19
15,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,yellow_tripdata_2010-06.csv,NaN,NaN
92,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,NaN,yellow_tripdata_2016-11.csv,NaN,NaN
108,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,NaN,yellow_tripdata_2018-03.csv,NaN,NaN
17,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,yellow_tripdata_2010-08.csv,NaN,NaN
20,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,yellow_tripdata_2010-11.csv,NaN,NaN


In [107]:
dfc = dfc[["file_name"] + [i for i in range(0,20)]] # uporzadkuj kolumny

# połącz nazwy oklumn w jeden string aby moc je pogrupować. 
dfc["key_cols"] = dfc[[i for i in range(0,20)]].apply(lambda x: ", ".join([str(x[i]) for i in range(0,20)]), axis=1)
dfc.sample(3)

,file_name,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,key_cols
62,yellow_tripdata_2014-05.csv,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,NaN,NaN,"vendor_id, pickup_datetime, dropoff_datetime..."
0,yellow_tripdata_2009-01.csv,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,...,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt,NaN,NaN,"vendor_name, Trip_Pickup_DateTime, Trip_Dropof..."
95,yellow_tripdata_2017-02.csv,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,NaN,NaN,NaN,"VendorID, tpep_pickup_datetime, tpep_dropoff_d..."


In [153]:
# #rozwiazanie nie potrzebne
# # zostawiam na widoku w razie potrzeby :)

# #dzięki pogrupowaniu dostajemy listę plików do których pasuje dany wzorzec nazw kolumn
# dfcg = dfc.groupby(["key_cols"], as_index=False)["file_name"].apply(','.join)
# dfcg["file_name"] = dfcg["file_name"].str.split(",")
# dfcg

,key_cols,file_name
0,"Unnamed: 0, VendorID, tpep_pickup_datetime, tp...","[yellow_tripdata_2015-01.csv, yellow_tripdata_..."
1,"Unnamed: 0, VendorID, tpep_pickup_datetime, tp...","[yellow_tripdata_2015-07.csv, yellow_tripdata_..."
2,"VendorID, tpep_pickup_datetime, tpep_dropoff_d...","[yellow_tripdata_2019-01.csv, yellow_tripdata_..."
3,"VendorID, tpep_pickup_datetime, tpep_dropoff_d...","[yellow_tripdata_2016-07.csv, yellow_tripdata_..."
4,"VendorID, tpep_pickup_datetime, tpep_dropoff_d...","[yellow_tripdata_2016-01.csv, yellow_tripdata_..."
5,"vendor_id, pickup_datetime, dropoff_datetime...","[yellow_tripdata_2014-01.csv, yellow_tripdata_..."
6,"vendor_id, pickup_datetime, dropoff_datetime, ...","[yellow_tripdata_2010-01.csv, yellow_tripdata_..."
7,"vendor_name, Trip_Pickup_DateTime, Trip_Dropof...","[yellow_tripdata_2009-01.csv, yellow_tripdata_..."


In [350]:
#utworzmy uniwersalne mapowanie kolumn
col_name_unique = ",".join(dfcg.key_cols.values).split(",")
col_name_unique = [x.strip() for x in col_name_unique]
col_name_unique = list(set(col_name_unique))   # set zostawi same unikalne nazwy
df_replace = pd.DataFrame(col_name_unique, columns=["old"])
df_replace["new"] = ""
df_replace[:3]

,old,new
0,mta_tax,
1,extra,
2,pickup_longitude,


In [348]:
#fukncja oczyszcza nazwe kolumny i odszukuje wzorców
def replace_column(old):
    old = " ".join(old.strip().lower().split("_"))
    
    if re.search(r'(?=.*vendor)', old):
        return "vendor"
    elif re.search(r'(tax)', old):
        return "tax"
    elif re.search(r'(?=.*rate)', old): 
        return "rate code" 
    elif re.search(r'(tolls)', old):
        return "tolls amount" 
    elif re.search(r'((?=.*pickup)(?=.*datetime))', old):
        return "pickup datetime" 
    elif re.search(r'((?=.*dropoff)(?=.*datetime))', old):
        return "dropoff datetime" 
    elif re.search(r'((?=.*start|.*pickup)(?=.*lat|.*latitude))', old):
        return "start latitude"
    elif re.search(r'((?=.*start|.*pickup)(?=.*lon|.*long|.*longitude))', old):
        return "start longitude" 
    elif re.search(r'((?=.*end|.*dropoff)(?=.*lat|.*latitude))', old):
        return "end latitude" 
    elif re.search(r'((?=.*end|.*dropoff)(?=.*lon|.*long|.*longitude))', old):
        return "end longitude"
    elif re.search(r'((?=.*store)(?=.*forward|.*fwd))', old):
        return "store forward" 
    elif re.search(r'((?=.*payment))', old):
        return "payment" 
    elif re.search(r'((?=.*extra))', old):
        return "extra"
    elif re.search(r'((?=.*trip)|(?=.*extra))', old):
        return "trip distance"
    elif re.search(r'((?=.*fare)(?=.*amount|.*amt))', old):
        return "fare amount"
    elif re.search(r'((?=.*tip)(?=.*amount|.*amt))', old):
        return "tip amount"
    elif re.search(r'((?=.*total)(?=.*amount|.*amt))', old):
        return "total amount" 
    elif re.search(r'(?=.*passenger)(?=.*count)', old):
        return "passenger count" 
    elif re.search(r'(surcharge)', old):
        return "surcharge" 
    elif re.search(r'(pulocationid)', old):
        return "pul location id" 
    elif re.search(r'(dolocationid)', old):
        return "dol location id" 
    else:
        return ""

df_replace["new"] = df_replace.apply(lambda x: replace_column(x["old"]), axis = 1)
df_replace

,old,new
0,mta_tax,tax
1,extra,extra
2,pickup_longitude,start longitude
3,Start_Lat,start latitude
4,rate_code,rate code
5,Payment_Type,payment
6,surcharge,surcharge
7,tolls_amount,tolls amount
8,fare_amount,fare amount
9,tip_amount,tip amount


In [290]:
# any(map(lambda x: x in [2,3,4], [1,2]))
# True
# any(map(lambda x: x in [2,3,4], [1,5]))
# False
# any(map(lambda x: x in [2,3,4], [2,4]))
# True


if re.search(r'(vendor|ve)', "vendor id"):
    print("jest")

hej


In [335]:
# if re.search(r'(?=.*vendor)(?=.*id)|(vendor)', "vendor"):
#     print("jest")
old = "store_and_fwd_flag"
old = " ".join(old.strip().lower().split("_"))
print(old)
if re.search(r'((?=.*store)(?=.*forward|.*fwd))', old):
    print("jest")

store and fwd flag
jest


In [313]:
list_sorted = [
 'VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, RateCodeID, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, Unnamed: 0',
 'VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, RatecodeID, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, Unnamed: 0',
 'VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, PULocationID, DOLocationID, nan, nan',
 'VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, PULocationID, DOLocationID, nan, nan, nan',
 'VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, RatecodeID, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, nan',
 'vendor_id,  pickup_datetime,  dropoff_datetime,  passenger_count,  trip_distance,  pickup_longitude,  pickup_latitude,  rate_code,  store_and_fwd_flag,  dropoff_longitude,  dropoff_latitude,  payment_type,  fare_amount,  surcharge,  mta_tax,  tip_amount,  tolls_amount,  total_amount, nan, nan',
 'vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, payment_type, fare_amount, surcharge, mta_tax, tip_amount, tolls_amount, total_amount, nan, nan',
 'vendor_name, Trip_Pickup_DateTime, Trip_Dropoff_DateTime, Passenger_Count, Trip_Distance, Start_Lon, Start_Lat, Rate_Code, store_and_forward, End_Lon, End_Lat, Payment_Type, Fare_Amt, surcharge, mta_tax, Tip_Amt, Tolls_Amt, Total_Amt, nan, nan'
]

In [61]:
dfc = dfc.drop_duplicates(keep="first", subset=[i for i in range(0,20)]) # wszystkie kolumny poza "file_name"
dfc

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,file,18,19
0,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,...,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt,yellow_tripdata_2009-01.csv,NaN,NaN
12,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,yellow_tripdata_2010-01.csv,NaN,NaN
58,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,yellow_tripdata_2014-01.csv,NaN,NaN
70,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,...,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,yellow_tripdata_2015-01.csv,improvement_surcharge,total_amount
76,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,...,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,yellow_tripdata_2015-07.csv,improvement_surcharge,total_amount
82,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,yellow_tripdata_2016-01.csv,total_amount,NaN
88,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,NaN,yellow_tripdata_2016-07.csv,NaN,NaN
118,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,yellow_tripdata_2019-01.csv,NaN,NaN


In [ ]:
# {
#     "2009": ['vendor_name', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'Rate_Code', 'store_and_forward', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt', 'surcharge', 'mta_tax', 'Tip_Amt', 'Tolls_Amt', 'Total_Amt'],
#     "2010": ['vendor_id', 'pickup_datetime',   'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
#     "2011": ['vendor_id', 'pickup_datetime',   'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
#     "2012": ['vendor_id', 'pickup_datetime',   'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
#     "2013": ['vendor_id', 'pickup_datetime',   'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
#     "2014": ['vendor_id', ' pickup_datetime', ' dropoff_datetime', ' passenger_count', ' trip_distance', ' pickup_longitude', ' pickup_latitude', ' rate_code', ' store_and_fwd_flag', ' dropoff_longitude', ' dropoff_latitude', ' payment_type', ' fare_amount', ' surcharge', ' mta_tax', ' tip_amount', ' tolls_amount', ' total_amount'],
#     "2015": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra', 'Unnamed: 0' ],
#     "2016": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra'],
#     "2017": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', ---------------  brak danych ------------ 'RatecodeID', 'store_and_fwd_flag',   --- zamiast koordyntów chyba na sektor---  'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra', 'PULocationID', 'DOLocationID'],
#     "2018": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', ---------------  brak danych ------------, 'RatecodeID', 'store_and_fwd_flag',   --- zamiast koordyntów chyba na sektor---  'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra', 'PULocationID', 'DOLocationID'],
#     "2019": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', ---------------  brak danych ------------, 'RatecodeID', 'store_and_fwd_flag',   --- zamiast koordyntów chyba na sektor---  'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra', 'PULocationID', 'DOLocationID', 'congestion_surcharge'],
#     "2020": ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', ---------------  brak danych ------------, 'RatecodeID', 'store_and_fwd_flag',   --- zamiast koordyntów chyba na sektor---  'payment_type', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'extra', 'PULocationID', 'DOLocationID', 'congestion_surcharge'],
# }


2015 -  nowa kolumna 'Unnamed: 0'
2015-2020 - nowa kolumna 'improvement_surcharge',  'extra' i 'congestion_surcharge'. Możliwe że 'extra' lub 'improvement_surcharge' to kolumna 'surcharge', która była w poprzednich latach
2017-2020   - zastąpienie  koordyntów początku i końca trasy (4 kolumny )  kolumnami 'PULocationID', 'DOLocationID' na określającymi sektor 


In [174]:
based_columns_clean = [clean_names(i) for i in based_columns]
print(based_columns_clean)

['vendor name', 'trip pickup datetime', 'trip dropoff datetime', 'passenger count', 'trip distance', 'start lon', 'start lat', 'rate code', 'store and forward', 'end lon', 'end lat', 'payment type', 'fare amt', 'surcharge', 'mta tax', 'tip amt', 'tolls amt', 'total amt']


In [171]:
def clean_names(x):
    x = x.strip()
    x = x.lower()
    x = x.split("_")
    x = " ".join(x)
    return(x)

In [172]:
for i in all_columns.keys():
    print(i)
    for z in all_columns[i]:
        z = clean_names(z)
        print(z)

yellow_tripdata_2009-01.csv
vendor name
trip pickup datetime
trip dropoff datetime
passenger count
trip distance
start lon
start lat
rate code
store and forward
end lon
end lat
payment type
fare amt
surcharge
mta tax
tip amt
tolls amt
total amt
yellow_tripdata_2009-02.csv
vendor name
trip pickup datetime
trip dropoff datetime
passenger count
trip distance
start lon
start lat
rate code
store and forward
end lon
end lat
payment type
fare amt
surcharge
mta tax
tip amt
tolls amt
total amt
yellow_tripdata_2009-03.csv
vendor name
trip pickup datetime
trip dropoff datetime
passenger count
trip distance
start lon
start lat
rate code
store and forward
end lon
end lat
payment type
fare amt
surcharge
mta tax
tip amt
tolls amt
total amt
yellow_tripdata_2009-04.csv
vendor name
trip pickup datetime
trip dropoff datetime
passenger count
trip distance
start lon
start lat
rate code
store and forward
end lon
end lat
payment type
fare amt
surcharge
mta tax
tip amt
tolls amt
total amt
yellow_tripdata_2009

improvement surcharge
total amount
yellow_tripdata_2016-07.csv
vendorid
tpep pickup datetime
tpep dropoff datetime
passenger count
trip distance
ratecodeid
store and fwd flag
pulocationid
dolocationid
payment type
fare amount
extra
mta tax
tip amount
tolls amount
improvement surcharge
total amount
yellow_tripdata_2016-08.csv
vendorid
tpep pickup datetime
tpep dropoff datetime
passenger count
trip distance
ratecodeid
store and fwd flag
pulocationid
dolocationid
payment type
fare amount
extra
mta tax
tip amount
tolls amount
improvement surcharge
total amount
yellow_tripdata_2016-09.csv
vendorid
tpep pickup datetime
tpep dropoff datetime
passenger count
trip distance
ratecodeid
store and fwd flag
pulocationid
dolocationid
payment type
fare amount
extra
mta tax
tip amount
tolls amount
improvement surcharge
total amount
yellow_tripdata_2016-10.csv
vendorid
tpep pickup datetime
tpep dropoff datetime
passenger count
trip distance
ratecodeid
store and fwd flag
pulocationid
dolocationid
payment

In [353]:
%%time
df = vaex.open('D:/TAXI/files_h5_two/yellow_tripdata_2015-07.csv.hdf5')
df

Wall time: 4.34 s


#,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,'2015-07-01 00:00:00','2015-07-01 00:15:26',1,3.5,-73.99415588378906,40.75112533569336,1,'N',-73.97682189941406,40.78856658935546,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3
1,1,1,'2015-07-01 00:00:00','2015-07-01 00:22:22',1,3.9,-73.98465728759764,40.76848602294922,1,'N',-74.00012969970702,40.73489761352539,2,17.0,0.5,0.5,0.0,0.0,0.3,18.3
2,2,1,'2015-07-01 00:00:00','2015-07-01 00:07:42',1,2.3,-73.97888946533203,40.76228713989258,1,'N',-74.00421905517578,40.752532958984375,2,9.0,0.5,0.5,0.0,0.0,0.3,10.3
3,3,1,'2015-07-01 00:00:00','2015-07-01 00:39:37',1,9.2,-73.99279022216798,40.74275970458984,1,'N',-73.9715118408203,40.63715362548828,1,33.0,0.5,0.5,8.55,0.0,0.3,42.85
4,4,1,'2015-07-01 00:00:00','2015-07-01 00:05:34',1,1.1,-73.9124298095703,40.76980972290039,1,'N',-73.92033386230467,40.757442474365234,1,6.0,0.5,0.5,2.0,0.0,0.3,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"11,562,778",11562778,1,'2015-07-02 13:05:48','2015-07-02 13:11:49',1,1.2,-73.9920883178711,40.75030899047852,1,'N',-73.97737884521484,40.75561141967773,1,6.5,0.0,0.5,1.46,0.0,0.3,8.76
"11,562,779",11562779,1,'2015-07-02 13:18:22','2015-07-02 13:24:45',2,1.8,-73.99122619628906,40.749954223632805,1,'N',-73.97188568115233,40.764286041259766,2,7.5,0.0,0.5,0.0,0.0,0.3,8.3
"11,562,780",11562780,1,'2015-07-02 13:36:18','2015-07-02 13:41:39',1,1.2,-73.99202728271484,40.749549865722656,1,'N',-73.97904205322266,40.75871276855469,1,6.0,0.0,0.5,1.7,0.0,0.3,8.5
"11,562,781",11562781,1,'2015-07-02 13:50:05','2015-07-02 13:54:01',1,0.8,-73.99166870117188,40.7491569519043,1,'N',-73.98018646240233,40.748359680175774,2,5.0,0.0,0.5,0.0,0.0,0.3,5.8


In [355]:
%%time
df = vaex.open('D:/TAXI/files_h5_two/yellow_tripdata_2015-01.csv.hdf5')
df

Wall time: 387 ms


#,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,'2015-01-15 19:05:39','2015-01-15 19:23:42',1,1.59,-73.993896484375,40.7501106262207,1,'N',-73.97478485107422,40.75061798095703,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,1,'2015-01-10 20:33:38','2015-01-10 20:53:28',1,3.3,-74.00164794921875,40.7242431640625,1,'N',-73.99441528320312,40.75910949707031,1,14.5,0.5,0.5,2.0,0.0,0.3,17.8
2,2,1,'2015-01-10 20:33:38','2015-01-10 20:43:41',1,1.8,-73.96334075927734,40.80278778076172,1,'N',-73.95182037353516,40.82441329956056,2,9.5,0.5,0.5,0.0,0.0,0.3,10.8
3,3,1,'2015-01-10 20:33:39','2015-01-10 20:35:31',1,0.5,-74.00908660888672,40.71381759643555,1,'N',-74.00432586669923,40.71998596191406,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
4,4,1,'2015-01-10 20:33:39','2015-01-10 20:52:58',1,3.0,-73.97117614746092,40.762428283691406,1,'N',-74.00418090820312,40.742652893066406,2,15.0,0.5,0.5,0.0,0.0,0.3,16.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"12,748,981",12748981,1,'2015-01-10 19:01:44','2015-01-10 19:05:40',2,1.0,-73.95198822021484,40.78621673583984,1,'N',-73.9537353515625,40.77516174316406,1,5.5,0.0,0.5,1.25,0.0,0.3,7.55
"12,748,982",12748982,1,'2015-01-10 19:01:44','2015-01-10 19:07:26',2,0.8,-73.9827423095703,40.72818374633789,1,'N',-73.9749755859375,40.72001266479492,1,6.0,0.0,0.5,2.0,0.0,0.3,8.8
"12,748,983",12748983,1,'2015-01-10 19:01:44','2015-01-10 19:15:01',1,3.4,-73.9793243408203,40.749549865722656,1,'N',-73.96910095214845,40.78779983520508,2,13.5,0.0,0.5,0.0,0.0,0.3,14.3
"12,748,984",12748984,1,'2015-01-10 19:01:44','2015-01-10 19:17:03',1,1.3,-73.99956512451173,40.73848342895508,1,'N',-73.98181915283203,40.737651824951165,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55


In [177]:
%%time
df = vaex.open('D:/TAXI/files_h5_two/yellow_tripdata_2017-01.csv.hdf5')
df

Wall time: 4.2 s


#,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,'2017-01-09 11:13:28','2017-01-09 11:25:45',1,3.3,1,'N',263,161,1,12.5,0.0,0.5,2.0,0.0,0.3,15.3
1,1,'2017-01-09 11:32:27','2017-01-09 11:36:01',1,0.9,1,'N',186,234,1,5.0,0.0,0.5,1.45,0.0,0.3,7.25
2,1,'2017-01-09 11:38:20','2017-01-09 11:42:05',1,1.1,1,'N',164,161,1,5.5,0.0,0.5,1.0,0.0,0.3,7.3
3,1,'2017-01-09 11:52:13','2017-01-09 11:57:36',1,1.1,1,'N',236,75,1,6.0,0.0,0.5,1.7,0.0,0.3,8.5
4,2,'2017-01-01 00:00:00','2017-01-01 00:00:00',1,0.02,2,'N',249,234,2,52.0,0.0,0.5,0.0,0.0,0.3,52.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"9,710,119",2,'2017-01-31 23:59:58','2017-02-01 00:20:01',1,4.95,1,'N',114,112,1,18.0,0.5,0.5,3.86,0.0,0.3,23.16
"9,710,120",2,'2017-01-31 23:59:58','2017-02-01 00:03:43',1,0.55,1,'N',114,113,1,4.0,0.5,0.5,0.5,0.0,0.3,5.8
"9,710,121",1,'2017-01-31 23:59:59','2017-02-01 00:24:35',2,5.0,1,'N',158,80,1,20.5,0.5,0.5,4.35,0.0,0.3,26.15
"9,710,122",2,'2017-01-31 23:59:59','2017-02-01 00:21:15',1,10.43,1,'N',138,162,1,30.5,0.5,0.5,5.0,5.54,0.3,42.34


In [3]:
## próba exportu do jednego pliku hdf5. 
## błąd spowodowany mają pamięcia ram. Może problem jest w bałaganie z columnami i ich typami


# %%time
# df.export('D:/TAXI/yellow_tripdata.hdf5')

KeyError: 'Passenger_Count'

`proba otwarcia i eksportu wielu plików naraz. `

In [50]:
df = vaex.open('test/yellow_taxi_*', header=None)
df

#,0,1,2,3,4,5,6,7,8,9
0,0.6916886110065336,0.740796620463854,0.23725536829970076,0.8935602252773415,0.9160105627800263,0.5263795590984541,0.8070804114194033,0.6247129471957079,0.5035602707462331,0.002476009978829463
1,0.6094545547131199,0.7580362660369044,0.278342959666942,0.6856480441144511,0.9218652835040739,0.5576004536135145,0.5321074585813166,0.9773181042063023,0.8334590442221034,0.05421793245762863
2,0.5559096119962542,0.21958416875034947,0.418554498407756,0.6309587422003907,0.5619034244768393,0.1880457165636081,0.9872641700259804,0.6913683383185752,0.16962860624701648,0.7705408221396257
3,0.5347047769326365,0.5373538494474387,0.49983921605455056,0.9944115334975159,0.3041165105596604,0.9220786210952069,0.21800835070029587,0.7953739014266499,0.4851836526480852,0.16022164212765133
4,0.6397803847918421,0.05850396021393345,0.44949726505780174,0.16635501340827075,0.916889160007984,0.5465982129651755,0.14568280325810112,0.9763553322667549,0.4655478105671346,0.7225059894348445
...,...,...,...,...,...,...,...,...,...,...
"19,332,923",0.5721229450586541,0.6732354978551108,0.5749287508714431,0.8798467069549151,0.6380985233012333,0.5065377950145137,0.6070996432143683,0.76854076327178,0.16657632412007106,0.9636453652719041
"19,332,924",0.9397467545673664,0.8337128146725512,0.41246811161037694,0.752464271984492,0.4455044080290576,0.8953377002397968,0.9198816480878106,0.39408385420010106,0.8397290515279605,0.14811902359340687
"19,332,925",0.8518432881907388,0.9575003051336006,0.2672360273141139,0.8690564276429411,0.09607723381993248,0.19805622349101004,0.6266320036132424,0.4841073228189722,0.8888731550902919,0.5078596764550816
"19,332,926",0.886198483835914,0.1536143550193075,0.87408843339827,0.9580789324743789,0.4141490997416383,0.7185782895451561,0.6477094387355473,0.48932570924954366,0.4873778607233673,0.20711298907925743


In [53]:
df.export_hdf5('yellow_taxi_big.hdf5')

In [54]:
df_h5_new = vaex.open('yellow_taxi_big.hdf5')
df_h5_new

#,0,1,2,3,4,5,6,7,8,9
0,0.6916886110065336,0.740796620463854,0.23725536829970076,0.8935602252773415,0.9160105627800263,0.5263795590984541,0.8070804114194033,0.6247129471957079,0.5035602707462331,0.002476009978829463
1,0.6094545547131199,0.7580362660369044,0.278342959666942,0.6856480441144511,0.9218652835040739,0.5576004536135145,0.5321074585813166,0.9773181042063023,0.8334590442221034,0.05421793245762863
2,0.5559096119962542,0.21958416875034947,0.418554498407756,0.6309587422003907,0.5619034244768393,0.1880457165636081,0.9872641700259804,0.6913683383185752,0.16962860624701648,0.7705408221396257
3,0.5347047769326365,0.5373538494474387,0.49983921605455056,0.9944115334975159,0.3041165105596604,0.9220786210952069,0.21800835070029587,0.7953739014266499,0.4851836526480852,0.16022164212765133
4,0.6397803847918421,0.05850396021393345,0.44949726505780174,0.16635501340827075,0.916889160007984,0.5465982129651755,0.14568280325810112,0.9763553322667549,0.4655478105671346,0.7225059894348445
...,...,...,...,...,...,...,...,...,...,...
"19,332,923",0.5721229450586541,0.6732354978551108,0.5749287508714431,0.8798467069549151,0.6380985233012333,0.5065377950145137,0.6070996432143683,0.76854076327178,0.16657632412007106,0.9636453652719041
"19,332,924",0.9397467545673664,0.8337128146725512,0.41246811161037694,0.752464271984492,0.4455044080290576,0.8953377002397968,0.9198816480878106,0.39408385420010106,0.8397290515279605,0.14811902359340687
"19,332,925",0.8518432881907388,0.9575003051336006,0.2672360273141139,0.8690564276429411,0.09607723381993248,0.19805622349101004,0.6266320036132424,0.4841073228189722,0.8888731550902919,0.5078596764550816
"19,332,926",0.886198483835914,0.1536143550193075,0.87408843339827,0.9580789324743789,0.4141490997416383,0.7185782895451561,0.6477094387355473,0.48932570924954366,0.4873778607233673,0.20711298907925743


###### proba połączenia dwóch df z roznymi jedna inną kolumna